In [ ]:
import numpy as np
import pandas as pd

import rasterio as rio

import matplotlib.pyplot as plt

root_file = './data/FuelsDataWBio/'
lf_25_file = './data/LF24_F40_250.ods'

pyromes= [
    31,33
]
eval_pyromes = [
    30
]
year = 2023
years = [
    # 2020,
    2021,
    2022,
    2023
]

from_vals = [
    91,92,93,98,99,
    101,102,103,104,105,106,107,108,109,
    121,122,123,124,
    141,142,143,144,145,146,147,148,149,
    161,162,163,164,165,
    181,182,183,184,185,186,187,188,189,
    201,202,203,204
]

to_vals = [
    1,1,1,1,1,
    2,2,2,2,2,2,2,2,2,
    3,3,3,3,
    4,4,4,4,4,4,4,4,4,
    5,5,5,5,5,
    6,6,6,6,6,6,6,6,6,
    7,7,7,7
]

lfdf = pd.read_excel(lf_25_file,engine='odf')

In [3]:
def load_data(pyromes,years):
    yearly_frames = []
    for pyrome in pyromes:
        for year in years:
            year_csv_file = root_file + f'stratified_sample_fbfm40_30m_{pyrome}_{year}.csv'
            year_fuels_sample = pd.read_csv(year_csv_file)
            yearly_frames.append(year_fuels_sample)

    fuels_frame = pd.concat(yearly_frames)

    return fuels_frame


def load_evt(year):
    if year == 2020:
        evt_file = f'./data/EVT/LF2020_EVT_220_CONUS/Tif/LC20_EVT_220.tif'
    elif year == 2022:
        evt_file = f'./data/EVT/LF2022_EVT_230_CONUS/Tif/LC22_EVT_230.tif'
    elif year == 2023:
        evt_file = f'./data/EVT/LF2023_EVT_240_CONUS/Tif/LC23_EVT_240.tif'
    evt_tif = rio.open(evt_file)

    return evt_tif

In [ ]:
evt = load_evt(2023)

In [9]:
evt.meta

{'driver': 'GTiff',
 'dtype': 'int16',
 'nodata': 32767.0,
 'width': 156335,
 'height': 101538,
 'count': 1,
 'crs': CRS.from_wkt('PROJCS["NAD83 / Conus Albers",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4269"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",23],PARAMETER["longitude_of_center",-96],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","5070"]]'),
 'transform': Affine(30.0, 0.0, -2362395.0,
        0.0, -30.0, 3267405.0)}

In [2]:
lfdf.head()

,VALUE,FBFM40,R,G,B,RED,GREEN,BLUE,sub_class,Fuel Load 1hr,...,Fuel Load 100hr,Fuel Load Live Herb,Fuel Load Live Woody,Fuel Model type,SAV ratio Dead 1hr,SAV Live herb,SAV Live Woody,fuel bed depth,dead fuel extinction moisture,heat content
0,-9999,Fill-NoData,255,255,255,1.000000,1.000000,1.000000,-9999,-1.0,...,-1.0,-1.0,-1.0,-1,-1,-1,-1,-1.0,-1,-1
1,91,NB1,104,104,104,0.407843,0.407843,0.407843,1,-1.0,...,-1.0,-1.0,-1.0,-1,-1,-1,-1,-1.0,-1,-1
2,92,NB2,225,225,225,0.882353,0.882353,0.882353,1,-1.0,...,-1.0,-1.0,-1.0,-1,-1,-1,-1,-1.0,-1,-1
3,93,NB3,255,237,237,1.000000,0.929412,0.929412,1,-1.0,...,-1.0,-1.0,-1.0,-1,-1,-1,-1,-1.0,-1,-1
4,98,NB8,0,14,214,0.000000,0.054902,0.839216,1,-1.0,...,-1.0,-1.0,-1.0,-1,-1,-1,-1,-1.0,-1,-1


In [3]:
burnable_categories = [2,3,4,5,6,7]

fuel_characteristics = [
    'Fuel Load 1hr', 'Fuel Load 10hr','Fuel Load 100hr', 'Fuel Load Live Herb','Fuel Load Live Woody',
    'SAV ratio Dead 1hr','SAV Live Herb','SAV Live Woody',
    'fuel bed depth',
    'dead fuel extinction moisture'
]

burnables_df = lfdf[lfdf['sub_class'].isin(burnable_categories)]